# Урок 9. Языковое моделирование

## Задание

Разобраться с моделькой генерации текста. <br>
Cобрать самим или взять датасет с вебинара и обучить генератор.

## Решение

 * Добавить отслежевание точности. 
 * Увеличить к-во эпох обучения. хотябы до 200.

In [2]:
import tensorflow as tf

import numpy as np
import os
import time

In [3]:
# Не показывать предупреждения.



In [4]:
path_to_file = 'evgenyi_onegin.txt'

In [5]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 286984 characters


In [6]:
print(text[:500])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высо


In [7]:
text = text + text

In [8]:
# The unique characters in the file
#  Отсортируем все символы в датасете и удалим повотряющиеся. 
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

131 unique characters


In [9]:
# Creating a mapping from unique characters to indices
#  Сделаем перевод номеров симоволов в тектст. 
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [10]:
# text_as_int, text, len(text_as_int), len(text)

In [11]:
text_as_int, text[:30], len(text_as_int), len(text)

(array([ 71, 110, 104, ..., 104, 121,   0]),
 'Александр Сергеевич Пушкин\n\n  ',
 573968,
 573968)

### train and target

In [12]:
# The maximum length sentence you want for a single input in characters

seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int) # ? Вот здесь не очень понятно. Зачем берут числа а не символы.

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

А
л
е
к
с


In [13]:
char_dataset.take(5)

<TakeDataset shapes: (), types: tf.int64>

In [14]:
# Выполним разбиение текста на последовательности длинной 100 символов.
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

# Выведем первые 5 батчей на экран.
for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '
'      Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        '
'Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        '
'Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Свят'
'ой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких д'


In [15]:
def split_input_target(chunk):
    input_text = chunk[:-1] # Предсказываем по одному символу.
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [16]:
dataset

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

Print the first example input and target values:

In [17]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '
Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


In [18]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [19]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 1024

In [20]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ]
)

In [21]:
class RNNgenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, batch_size):
        super(RNNgenerator, self).__init__()
        
        self.emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
                                 
        self.gru1 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
        self.gru2 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
                           
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        emb_x = self.emb(x)
        x1 = self.gru1(emb_x)
        x = x1
        for _ in range(3):
            x = self.gru2(x)
        #x = self.gru1(x)
        x = (x + x1)/2
        return self.fc(x)

model = RNNgenerator(vocab_size, embedding_dim, BATCH_SIZE)

In [22]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [23]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [24]:
model = build_model(
            vocab_size=len(vocab),
            embedding_dim=embedding_dim,
            rnn_units=rnn_units,
            batch_size=BATCH_SIZE
        )

In [25]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 128)         16768     
_________________________________________________________________
gru_2 (GRU)                  (None, None, 1024)        3545088   
_________________________________________________________________
gru_3 (GRU)                  (None, None, 1024)        6297600   
_________________________________________________________________
gru_4 (GRU)                  (None, None, 1024)        6297600   
_________________________________________________________________
dense_2 (Dense)              (None, None, 131)         134275    
Total params: 16,291,331
Trainable params: 16,291,331
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Посмотрим, что выдаст не тренированная сеть.
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 131) # (batch_size, sequence_length, vocab_size)


In [27]:
# Посотрим как выглядитя выходные значения из нейронной сети.

In [28]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 131), dtype=float32, numpy=
array([[ 3.7909424e-04, -3.6372356e-05,  9.0024777e-04, ...,
         7.1353788e-05,  1.0551377e-03,  3.6968770e-05],
       [ 6.0743577e-04, -1.9961620e-04,  2.2276253e-03, ...,
         2.6402494e-04,  2.8263552e-03, -1.8621341e-04],
       [ 5.7593192e-04, -4.0907520e-04,  3.6346342e-03, ...,
         5.1270227e-04,  4.7459309e-03, -7.5447571e-04],
       ...,
       [-4.7122405e-04, -2.1664561e-04,  8.5583609e-03, ...,
         1.1607811e-03,  1.0676588e-02, -4.3490971e-03],
       [-4.9212069e-04, -2.9403978e-04,  8.9309318e-03, ...,
         9.5256994e-04,  1.0989322e-02, -4.7025345e-03],
       [-4.9350440e-04, -3.3539938e-04,  9.1903666e-03, ...,
         7.6076289e-04,  1.1222585e-02, -4.9679456e-03]], dtype=float32)>

In [29]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [30]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 '                              XXXI\n\n                        Отъезда день давно просрочен,\n          '

Next Char Predictions: 
 'хz,ФyкЧ)3P.!Iж1{КVК5sуУШQuя0cПnB{BNCЕфЧЮВаб-вЕLзw}QзЧ{о9yйцМlFж}ifr2г,1L6ГR9v:ш2?i)WмwAюeA1Lм8MoCyфН'


## Train the model

In [31]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# example_batch_loss = loss(target_example_batch, example_batch_predictions)
# print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
# print("scalar_loss:      ", example_batch_loss.numpy().mean())

In [32]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [33]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    period=20,
    save_weights_only=True)

### Execute the training

In [34]:
EPOCHS = 3

In [35]:
history = model.fit(dataset,
                    epochs=EPOCHS,
                    callbacks=[checkpoint_callback]
                   )

Epoch 1/20
88/88 [==============================] - 47s 433ms/step - loss: 1.9722
Epoch 2/20
88/88 [==============================] - 39s 432ms/step - loss: 1.4598
Epoch 3/20
88/88 [==============================] - 38s 429ms/step - loss: 1.3050
Epoch 4/20
88/88 [==============================] - 38s 430ms/step - loss: 1.2218
Epoch 5/20
88/88 [==============================] - 39s 433ms/step - loss: 1.1336
Epoch 6/20
88/88 [==============================] - 39s 431ms/step - loss: 1.0468
Epoch 7/20
88/88 [==============================] - 39s 432ms/step - loss: 0.9553
Epoch 8/20
88/88 [==============================] - 38s 430ms/step - loss: 0.8882
Epoch 9/20
88/88 [==============================] - 38s 431ms/step - loss: 0.7828
Epoch 10/20
88/88 [==============================] - 38s 432ms/step - loss: 0.6851
Epoch 11/20
88/88 [==============================] - 38s 431ms/step - loss: 0.5975
Epoch 12/20
88/88 [==============================] - 39s 432ms/step - loss: 0.4909
Epoch 13/20
8

## Generate text

In [36]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_20'

In [37]:
# Востановим модель из кеша.

In [38]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [39]:
# model.summary()

In [40]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [41]:
text_ = generate_text(model, start_string=u"И вот идет уже ")
print(text_)

И вот идет уже идост                        рось.      пи и
  увей    ие  Сваста      зна       примузиводраскобре,                   Та,   "      сливидякамнемий                    Ит"  Лей ряели закора      -
    Амне
                 ряпочь;
    и,         недить В
      было
      бы,     Вожа,            вот вак  Брузнабрах           Xн?.  здо   дугел          че,
 и          блI
    Пей        Гавух сторая  Ле  Петоска содевораканеть        Чт         Xак гулаловоюн      сть,      питнечьсос     .       
